In [1]:
import retro
import time
import numpy as np
from gym import Env
from gym.spaces import MultiBinary, Box, Dict
import cv2
from matplotlib import pyplot as plt
import os
import sys
import optuna
from stable_baselines3 import PPO
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv, VecFrameStack
from stable_baselines3.common.callbacks import BaseCallback
from stable_baselines3.common.env_checker import check_env

In [2]:
!python -m retro.import ./custom_integrations

Importing StreetFighterIISpecialChampionEdition-Genesis
Imported 1 games


In [3]:
class StreetFighter(Env): 
    def __init__(self):
        super().__init__()
        # Specify action space and observation space 
        self.observation_space = Box(low=0, high=255, shape=(84, 84, 1), dtype=np.uint8)
        self.action_space = MultiBinary(12)
        # Startup and instance of the game 
        self.game = retro.make(game='StreetFighterIISpecialChampionEdition-Genesis', use_restricted_actions=retro.Actions.FILTERED)
    
    def reset(self):
        # Return the first frame 
        obs = self.game.reset()
        obs = self.preprocess(obs) 
        self.previous_frame = obs 
        
        # Create a attribute to hold the score delta 
        self.score = 0 
        return obs
    
    def preprocess(self, observation): 
        # Grayscaling 
        gray = cv2.cvtColor(observation, cv2.COLOR_BGR2GRAY)
        # Resize 
        resize = cv2.resize(gray, (84,84), interpolation=cv2.INTER_CUBIC)
        # Add the channels value
        channels = np.reshape(resize, (84,84,1))
        return channels 
    
    def step(self, action): 
        # Take a step 
        obs, reward, done, info = self.game.step(action)
        obs = self.preprocess(obs) 
        
        # Frame delta 
        frame_delta = obs - self.previous_frame
        self.previous_frame = obs 
        
        # Reshape the reward function
        reward = info['score'] - self.score 
        self.score = info['score'] 
        
        return frame_delta, reward, done, info
    
    def render(self, *args, **kwargs):
        self.game.render()
        
    def close(self):
        self.game.close()

In [4]:
LOG_DIR = './logs/PPO/'
OPT_DIR = './opt/PPO/'

In [5]:
def optimize_ppo(trial): 
    return {
        'n_steps':trial.suggest_int('n_steps', 2048, 8192),
        'gamma':trial.suggest_loguniform('gamma', 0.8, 0.9999),
        'learning_rate':trial.suggest_loguniform('learning_rate', 1e-5, 1e-4),
        'clip_range':trial.suggest_uniform('clip_range', 0.1, 0.4),
        'gae_lambda':trial.suggest_uniform('gae_lambda', 0.8, 0.99)
    }

In [6]:
SAVE_PATH = os.path.join(OPT_DIR, 'trial_{}_best_model'.format(1))

In [7]:
def optimize_agent(trial):
    try:
        model_params = optimize_ppo(trial) 

        # Create environment 
        env = StreetFighter()
        env = Monitor(env, LOG_DIR)
        env = DummyVecEnv([lambda: env])
        env = VecFrameStack(env, 4, channels_order='last')

        # Create algo 
        model = PPO('CnnPolicy', env, tensorboard_log=LOG_DIR, verbose=0, **model_params)
        model.learn(total_timesteps=80000, progress_bar=True)
        #model.learn(total_timesteps=100000)

        # Evaluate model 
        mean_reward, _ = evaluate_policy(model, env, n_eval_episodes=5)
        env.close()

        SAVE_PATH = os.path.join(OPT_DIR, 'trial_{}_best_model'.format(trial.number))
        model.save(SAVE_PATH)

        return mean_reward

    except Exception as e:
        print(e)
        return -1000

In [8]:
study = optuna.create_study(direction='maximize')
study.optimize(optimize_agent, n_trials=10, n_jobs=1)

[I 2024-04-20 13:49:02,153] A new study created in memory with name: no-name-fde852e6-143d-4683-a186-5a6eb27147ce


Output()

c:\Users\TheHeatbringer\Anaconda3\envs\RLProject\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  after removing the cwd from sys.path.
c:\Users\TheHeatbringer\Anaconda3\envs\RLProject\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  """
c:\Users\TheHeatbringer\Anaconda3\envs\RLProject\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  
c:\Users\TheHeatbringer\Anaconda3\envs\RLProject\lib\site-packages\ipykernel_l

[W 2024-04-20 13:49:09,647] Trial 0 failed with parameters: {'n_steps': 4290, 'gamma': 0.9744923434473374, 'learning_rate': 2.9737802398534057e-05, 'clip_range': 0.20299728519599666, 'gae_lambda': 0.9357898835902296} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "c:\Users\TheHeatbringer\Anaconda3\envs\RLProject\lib\site-packages\optuna\study\_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\TheHeatbringer\AppData\Local\Temp\ipykernel_20100\2980155279.py", line 13, in optimize_agent
    model.learn(total_timesteps=80000, progress_bar=True)
  File "c:\Users\TheHeatbringer\Anaconda3\envs\RLProject\lib\site-packages\stable_baselines3\ppo\ppo.py", line 314, in learn
    progress_bar=progress_bar,
  File "c:\Users\TheHeatbringer\Anaconda3\envs\RLProject\lib\site-packages\stable_baselines3\common\on_policy_algorithm.py", line 250, in learn
    continue_training = self.collect_rollouts(self.env, callback, 

KeyboardInterrupt: 

In [9]:
model = PPO.load(os.path.join(OPT_DIR, 'trial_8_best_model.zip'))

In [10]:
class TrainAndLoggingCallback(BaseCallback):

    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path

    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'best_model_{}'.format(self.n_calls))
            self.model.save(model_path)

        return True

In [11]:
CHECKPOINT_DIR = './train/PPO/'

In [12]:
callback = TrainAndLoggingCallback(check_freq=10000, save_path=CHECKPOINT_DIR)

In [13]:
env = StreetFighter()
env = Monitor(env, LOG_DIR)
env = DummyVecEnv([lambda: env])
env = VecFrameStack(env, 4, channels_order='last')

In [16]:
model_params = study.best_params
model_params['n_steps'] = 3584  # set n_steps to 7488 or a factor of 64
# model_params['learning_rate'] = 5e-7
model = PPO('CnnPolicy', env, tensorboard_log=LOG_DIR, verbose=1, **model_params)
model.load(os.path.join(OPT_DIR, 'trial_8_best_model.zip'))

Using cuda device
Wrapping the env in a VecTransposeImage.


In [18]:
model.learn(total_timesteps=1000000, callback=callback, progress_bar=True)

Logging to ./logs/PPO/PPO_13


Output()

-----------------------------
| time/              |      |
|    fps             | 376  |
|    iterations      | 1    |
|    time_elapsed    | 9    |
|    total_timesteps | 3584 |
-----------------------------


------------------------------------------
| time/                   |              |
|    fps                  | 264          |
|    iterations           | 2            |
|    time_elapsed         | 27           |
|    total_timesteps      | 7168         |
| train/                  |              |
|    approx_kl            | 0.0021263584 |
|    clip_fraction        | 0.085        |
|    clip_range           | 0.126        |
|    entropy_loss         | -8.31        |
|    explained_variance   | -0.000362    |
|    learning_rate        | 1.8e-05      |
|    loss                 | 21.3         |
|    n_updates            | 20           |
|    policy_gradient_loss | -0.00143     |
|    value_loss           | 2.68e+03     |
------------------------------------------


-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 1.01e+04    |
|    ep_rew_mean          | 1.18e+04    |
| time/                   |             |
|    fps                  | 247         |
|    iterations           | 3           |
|    time_elapsed         | 43          |
|    total_timesteps      | 10752       |
| train/                  |             |
|    approx_kl            | 0.002087639 |
|    clip_fraction        | 0.0755      |
|    clip_range           | 0.126       |
|    entropy_loss         | -8.31       |
|    explained_variance   | 0.000103    |
|    learning_rate        | 1.8e-05     |
|    loss                 | 171         |
|    n_updates            | 30          |
|    policy_gradient_loss | -0.00178    |
|    value_loss           | 7.53e+03    |
-----------------------------------------


...

------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 6.71e+03     |
|    ep_rew_mean          | 1.03e+04     |
| time/                   |              |
|    fps                  | 216          |
|    iterations           | 276          |
|    time_elapsed         | 4562         |
|    total_timesteps      | 989184       |
| train/                  |              |
|    approx_kl            | 0.0024986963 |
|    clip_fraction        | 0.0696       |
|    clip_range           | 0.126        |
|    entropy_loss         | -0.594       |
|    explained_variance   | 0.208        |
|    learning_rate        | 1.8e-05      |
|    loss                 | 4.2          |
|    n_updates            | 2760         |
|    policy_gradient_loss | 0.00139      |
|    value_loss           | 262          |
------------------------------------------


-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 6.68e+03    |
|    ep_rew_mean          | 9.88e+03    |
| time/                   |             |
|    fps                  | 216         |
|    iterations           | 277         |
|    time_elapsed         | 4578        |
|    total_timesteps      | 992768      |
| train/                  |             |
|    approx_kl            | 0.003227774 |
|    clip_fraction        | 0.06        |
|    clip_range           | 0.126       |
|    entropy_loss         | -0.571      |
|    explained_variance   | 0.0972      |
|    learning_rate        | 1.8e-05     |
|    loss                 | 3.16e+03    |
|    n_updates            | 2770        |
|    policy_gradient_loss | -0.000394   |
|    value_loss           | 1.58e+03    |
-----------------------------------------


------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 6.6e+03      |
|    ep_rew_mean          | 9.58e+03     |
| time/                   |              |
|    fps                  | 216          |
|    iterations           | 278          |
|    time_elapsed         | 4594         |
|    total_timesteps      | 996352       |
| train/                  |              |
|    approx_kl            | 0.0060732877 |
|    clip_fraction        | 0.107        |
|    clip_range           | 0.126        |
|    entropy_loss         | -0.652       |
|    explained_variance   | 0.0985       |
|    learning_rate        | 1.8e-05      |
|    loss                 | 3.9          |
|    n_updates            | 2780         |
|    policy_gradient_loss | 0.00094      |
|    value_loss           | 795          |
------------------------------------------


------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 6.56e+03     |
|    ep_rew_mean          | 9.11e+03     |
| time/                   |              |
|    fps                  | 216          |
|    iterations           | 279          |
|    time_elapsed         | 4610         |
|    total_timesteps      | 999936       |
| train/                  |              |
|    approx_kl            | 0.0034698057 |
|    clip_fraction        | 0.0689       |
|    clip_range           | 0.126        |
|    entropy_loss         | -0.648       |
|    explained_variance   | 0.154        |
|    learning_rate        | 1.8e-05      |
|    loss                 | 15.1         |
|    n_updates            | 2790         |
|    policy_gradient_loss | 0.00049      |
|    value_loss           | 773          |
------------------------------------------


------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 6.56e+03     |
|    ep_rew_mean          | 9.11e+03     |
| time/                   |              |
|    fps                  | 216          |
|    iterations           | 280          |
|    time_elapsed         | 4627         |
|    total_timesteps      | 1003520      |
| train/                  |              |
|    approx_kl            | 0.0058250693 |
|    clip_fraction        | 0.091        |
|    clip_range           | 0.126        |
|    entropy_loss         | -0.623       |
|    explained_variance   | 0.13         |
|    learning_rate        | 1.8e-05      |
|    loss                 | 428          |
|    n_updates            | 2800         |
|    policy_gradient_loss | 0.00252      |
|    value_loss           | 1.72e+03     |
------------------------------------------


In [25]:
model = PPO.load('./train/PPO/best_model_250000.zip')
mean_reward, _ = evaluate_policy(model, env, render=True, n_eval_episodes=5)
mean_reward

3200.0

: 

In [24]:
obs = env.reset()
# Set flag to flase
done = False
for game in range(1): 
    while not done: 
        if done: 
            obs = env.reset()
        env.render()
        action = model.predict(obs)[0]
        obs, reward, done, info = env.step(action)
        time.sleep(0.005)
        print(reward)

[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
